In [1]:
# matplotlib plots within notebook
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np

from datetime import datetime
from tqdm import tqdm

import os, sys

from l5kit.configs import load_config_data
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import AgentDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv
from l5kit.geometry import transform_points


# Custom libs
sys.path.insert(0, './LyftAgent_lib')
from LyftAgent_lib import train_support as lyl_ts
from LyftAgent_lib import topologies as lyl_nn

# Print Code Version
import git
def print_git_info(path, nombre):
    repo = git.Repo(path)
    print('Using: %s \t branch %s \t commit hash %s'%(nombre, repo.active_branch.name, repo.head.object.hexsha))
    changed = [ item.a_path for item in repo.index.diff(None) ]
    if len(changed)>0:
        print('\t\t WARNING -- modified files:')
        print(changed)
    
print_git_info('.', 'LyftAgent_lib')


import platform
print("python: "+platform.python_version())


import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
print('Using TensorFlow version: '+tf.__version__)
print('Using Keras version: '+keras.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using: LyftAgent_lib 	 branch master 	 commit hash df7c2d9ee4ec0471290a5590b9d6274611bc223c
		 WARNING -- modified files:
['AgentPrediction_Train.ipynb', 'AgentPrediction_config.yaml', 'AgentPrediction_config_Baseline.yaml', 'LyftAgent_lib/train_support.py']
python: 3.6.9
Using TensorFlow version: 2.3.1
Using Keras version: 2.4.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6266086559033967116
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4701137181891413462
physical_device_desc: "device: XLA_CPU device"
]


# Definitions and configuration

In [2]:
# Test model loction
path_load = ''
# Test model base name
net_base_name = ''

# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = ""
# get config
cfg = load_config_data("./AgentPrediction_config.yaml")
cfg = lyl_ts.fill_defaults(cfg)

# Validation chopped dataset:
eval_base_path = ''


In [3]:

model_map_input_shape      = (cfg["raster_params"]["raster_size"][0],
                              cfg["raster_params"]["raster_size"][1])
num_hist_frames            = cfg["model_params"]["history_num_frames"]
num_future_frames          = cfg["model_params"]["future_num_frames"]

increment_net              = cfg["model_params"]["increment_net"]

mruv_guiding               = cfg["model_params"]["mruv_guiding"]
mruv_model_trainable       = cfg["model_params"]["mruv_model_trainable"]

retrain_inputs_image_model = cfg["training_params"]["retrain_inputs_image_model"]
gen_batch_size             = cfg["train_data_loader"]["batch_size"]


model_version              = cfg["model_params"]["version"]
base_image_preprocess      = cfg["model_params"]["base_image_preprocess"]
use_fading                 = cfg["model_params"]["use_fading"]

use_angle                  = cfg["model_params"]["use_angle"]

isBaseModel = False
if model_version == 'Base':
    isBaseModel = True
    forward_pass_use = lyl_nn.modelBaseline_forward_pass
elif model_version == 'V1':
    forward_pass_use = lyl_nn.modelV1_forward_pass
elif model_version == 'V2':
    forward_pass_use = lyl_nn.modelV2_forward_pass
    

# Get image preprocessing function (depends on image encoding base architecture)
if base_image_preprocess == None:
    base_image_preprocess_fcn = lambda x: x
else:
    try:
        base_image_preprocess_fcn = getattr(keras.applications, base_image_preprocess.split('.')[0])
        base_image_preprocess_fcn = getattr(base_image_preprocess_fcn, base_image_preprocess.split('.')[1])
    except:
        raise Exception('Base image pre-processing not found. Requested function: %s'%base_image_preprocess) 

### Load model

In [5]:

model_list = lyl_ts.load_models(path_load, net_base_name, 
                         load_img_model = (retrain_inputs_image_model or retrain_all_image_model), 
                         isBaseModel = isBaseModel,
                         mruv_guiding = mruv_guiding)

ImageEncModel = model_list[0]
HistEncModel = model_list[1]
PathDecModel = model_list[2]
mruv_model = model_list[3]


All models succesfully loaded.


### Load dataset

In [15]:
dm = LocalDataManager()
rast = build_rasterizer(cfg, dm)

dataset_path_test = dm.require(cfg["test_data_loader"]["key"])
test_zarr = ChunkedDataset(dataset_path_test)
test_zarr.open()
print(test_zarr)

test_mask = np.load(f"../prediction-dataset/scenes/mask.npz")["arr_0"]

test_dataset = AgentDataset(cfg, test_zarr, rast, agents_mask=test_mask)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   11314    |  1131400   |  88594921  |    7854144    |      31.43      |        100.00        |        78.31         |        10.00         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: you're running with a custom agents_mask
  # This is added back by InteractiveShellApp.in

In [18]:
tf_test_dataset = lyl_ts.get_tf_dataset(test_dataset, 
                                             num_hist_frames,
                                             model_map_input_shape,
                                             num_future_frames,
                                             meta_dict_use = lyl_ts.meta_dict_pass)

# Map sample pre-processing function
tf_test_dataset = tf_test_dataset.map(lambda x: lyl_ts.tf_get_input_sample(x, 
                                                                             image_preprocess_fcn=base_image_preprocess_fcn, 
                                                                             use_fading = use_fading,
                                                                             use_angle = use_angle))
# Set batch size
tf_test_dataset = tf_test_dataset.batch(batch_size=gen_batch_size)

    

Creating dataset with: 
	 Randomized scenes: False
	 Randomized frames: False
	 Number of scenes: 11314
	 Number of frames per scenes: -1


# Test!

In [24]:
future_coords_offsets_pd = []
timestamps = []
agent_ids = []



test_dataset_prog_bar = tqdm(tf_test_dataset, total=int(np.ceil(len(test_dataset)/gen_batch_size)))
test_dataset_prog_bar.set_description('Testing: ')
for (thisSampleMapComp, thisSampeHistPath, thisSampeTargetPath, 
    thisHistAvail, thisTargetAvail, 
    thisTimeStamp, thisTrackID, thisRasterFromAgent, thisWorldFromAgent, thisCentroid, thisSampleIdx) in test_dataset_prog_bar:
    
    pad_size = gen_batch_size-thisSampleMapComp.shape[0]
    if pad_size != 0:
        # Create padded batch if necessary
        aux_SampleMapComp = np.zeros((gen_batch_size,thisSampleMapComp.shape[1],thisSampleMapComp.shape[2],thisSampleMapComp.shape[3]), dtype = np.float32)
        aux_SampeHistPath = np.zeros((gen_batch_size,thisSampeHistPath.shape[1],thisSampeHistPath.shape[2]), dtype = np.float32)
        aux_SampeTargetPath = np.zeros((gen_batch_size,thisSampeTargetPath.shape[1],thisSampeTargetPath.shape[2]), dtype = np.float32)
        aux_HistAvail = np.zeros((gen_batch_size,thisHistAvail.shape[1]), dtype = np.float32)
        aux_TargetAvail = np.zeros((gen_batch_size,thisTargetAvail.shape[1]), dtype = np.float32)
        aux_TimeStamp = np.zeros((gen_batch_size), dtype = np.float32)
        aux_TrackID = np.zeros((gen_batch_size), dtype = np.float32)

        aux_SampleMapComp[:gen_batch_size-pad_size,:,:,:] = thisSampleMapComp
        aux_SampeHistPath[:gen_batch_size-pad_size,:,:] = thisSampeHistPath
        aux_SampeTargetPath[:gen_batch_size-pad_size,:,:] = thisSampeTargetPath
        aux_HistAvail[:gen_batch_size-pad_size,:] = thisHistAvail
        aux_TargetAvail[:gen_batch_size-pad_size,:] = thisTargetAvail
        aux_TimeStamp[:gen_batch_size-pad_size] = thisTimeStamp
        aux_TrackID[:gen_batch_size-pad_size] = thisTrackID

        thisSampleMapComp = aux_SampleMapComp
        thisSampeHistPath = aux_SampeHistPath
        thisSampeTargetPath = aux_SampeTargetPath
        thisHistAvail = aux_HistAvail
        thisTargetAvail = aux_TargetAvail
        TimeStamp = aux_TimeStamp
        thisTrackID = aux_TrackID
        
        
        
    # Predict
    if base_model:
        predPath = forwardpass_use(thisSampleMapComp, ImageEncModel,  PathDecModel)
    else:
        PathDecModel.reset_states()
        HistEncModel.reset_states()
        predPath = forwardpass_use(thisSampleMapComp, thisSampeHistPath, thisHistAvail, 50,
                                                ImageEncModel, HistEncModel, PathDecModel,
                                                use_teacher_force=False,
                                                increment_net = increment_net,
                                                mruv_guiding = mruv_guiding,
                                                mruv_model = mruv_model,
                                                mruv_model_trainable = mruv_model_trainable)

    predPath = predPath.numpy()

    # convert agent coordinates into world offsets
    predPath = predPath[:,:,:2]
    world_from_agents = thisWorldFromAgent.numpy()
    centroids = thisCentroid.numpy()



    
    for idx_sample in range(gen_batch_size-pad_size):
        
        # Save info
        future_coords_offsets_pd.append(transform_points(predPath[idx_sample,:,:], thisWorldFromAgent.numpy()[idx_sample,:,:]) -thisCentroid.numpy()[idx_sample,:])
        timestamps.append(thisTimeStamp[idx_sample].numpy())
        agent_ids.append(thisTrackID[idx_sample])

assert len(agent_ids) == len(test_dataset), "Test data size not equal to dataset."
    

Testing: :  19%|█▉        | 417/2223 [13:25<58:06,  1.93s/it]  


KeyboardInterrupt: 

### Write result csv

    Encode the predictions into a csv file. Coords can have an additional axis for multi-mode.
    We handle up to MAX_MODES modes.
    For the uni-modal case (i.e. all predictions have just a single mode), coords should not have the additional axis
    and confs should be set to None. In this case, a single mode with confidence 1 will be written.
    Args:
        csv_path (str): path to the csv to write
        timestamps (np.ndarray): (num_example,) frame timestamps
        track_ids (np.ndarray): (num_example,) agent ids
        coords (np.ndarray): (num_example x (modes) x future_len x num_coords) meters displacements
        confs (Optional[np.ndarray]): (num_example x modes) confidence of each modes in each example.
        Rows should sum to 1
    Returns:


In [14]:
write_pred_csv('submission.csv',
               timestamps=np.array(timestamps),
               track_ids=np.array(agent_ids),
               coords=np.array(future_coords_offsets_pd),
              confs=None)

ValueError: zero-dimensional arrays cannot be concatenated

# Validation 

This will perform validation against a chopped dataset. It will provide a value near the test value of the leader board.

In [6]:
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace, rmse, average_displacement_error_mean


In [7]:
dm_val = LocalDataManager()

# ===== GENERATE AND LOAD CHOPPED DATASET
num_frames_to_chop = 100
eval_cfg = cfg["val_data_loader"]
if not os.path.exists(eval_base_path):
    eval_base_path = create_chopped_dataset(dm_val.require(eval_cfg["key"]), 
                                            cfg["raster_params"]["filter_agents_threshold"], 
                                            num_frames_to_chop,
                                            cfg["model_params"]["future_num_frames"],
                                            MIN_FUTURE_STEPS)

In [8]:
rast_val = build_rasterizer(cfg, dm_val)


eval_zarr_path = os.path.join(eval_base_path, 'validate.zarr')
eval_mask_path = os.path.join(eval_base_path, 'mask.npz') 
eval_gt_path =  os.path.join(eval_base_path, 'gt.csv') 

eval_zarr = ChunkedDataset(eval_zarr_path).open()
eval_mask = np.load(eval_mask_path)["arr_0"]
# ===== INIT DATASET AND LOAD MASK
validation_dataset = AgentDataset(cfg, eval_zarr, rast_val, agents_mask=eval_mask)
# eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
#                              num_workers=eval_cfg["num_workers"])
print(validation_dataset)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: you're running with a custom agents_mask
  # This is added back by InteractiveShellApp.init_path()
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16220    |  1622000   | 125423254  |    11733321   |      45.06      |        100.00        |        77.33         |        10.00         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+-------------

In [9]:

tf_validation_dataset = lyl_ts.get_tf_dataset(validation_dataset, 
                                             num_hist_frames,
                                             model_map_input_shape,
                                             num_future_frames,
                                             meta_dict_use = lyl_ts.meta_dict_pass)

# Map sample pre-processing function
tf_validation_dataset = tf_validation_dataset.map(lambda x: lyl_ts.tf_get_input_sample(x, 
                                                                             image_preprocess_fcn=base_image_preprocess_fcn, 
                                                                             use_fading = use_fading,
                                                                             use_angle = use_angle))
# Set batch size
tf_validation_dataset = tf_validation_dataset.batch(batch_size=gen_batch_size)

    

Creating dataset with: 
	 Randomized scenes: False
	 Randomized frames: False
	 Number of scenes: 16220
	 Number of frames per scenes: -1


In [10]:
future_coords_offsets_pd = []
timestamps = []
agent_ids = []

val_dataset_prog_bar = tqdm(tf_validation_dataset, total=int(np.ceil(len(validation_dataset)/gen_batch_size)))
val_dataset_prog_bar.set_description('Validating: ')
for (thisSampleMapComp, thisSampeHistPath, thisSampeTargetPath, 
    thisHistAvail, thisTargetAvail, 
    thisTimeStamp, thisTrackID, thisRasterFromAgent, thisWorldFromAgent, thisCentroid, thisSampleIdx) in val_dataset_prog_bar:
    
    pad_size = gen_batch_size-thisSampleMapComp.shape[0]
    if pad_size != 0:
        # Create padded batch if necessary
        aux_SampleMapComp = np.zeros((gen_batch_size,thisSampleMapComp.shape[1],thisSampleMapComp.shape[2],thisSampleMapComp.shape[3]), dtype = np.float32)
        aux_SampeHistPath = np.zeros((gen_batch_size,thisSampeHistPath.shape[1],thisSampeHistPath.shape[2]), dtype = np.float32)
        aux_SampeTargetPath = np.zeros((gen_batch_size,thisSampeTargetPath.shape[1],thisSampeTargetPath.shape[2]), dtype = np.float32)
        aux_HistAvail = np.zeros((gen_batch_size,thisHistAvail.shape[1]), dtype = np.float32)
        aux_TargetAvail = np.zeros((gen_batch_size,thisTargetAvail.shape[1]), dtype = np.float32)
        aux_TimeStamp = np.zeros((gen_batch_size), dtype = np.float32)
        aux_TrackID = np.zeros((gen_batch_size), dtype = np.float32)

        aux_SampleMapComp[:gen_batch_size-pad_size,:,:,:] = thisSampleMapComp
        aux_SampeHistPath[:gen_batch_size-pad_size,:,:] = thisSampeHistPath
        aux_SampeTargetPath[:gen_batch_size-pad_size,:,:] = thisSampeTargetPath
        aux_HistAvail[:gen_batch_size-pad_size,:] = thisHistAvail
        aux_TargetAvail[:gen_batch_size-pad_size,:] = thisTargetAvail
        aux_TimeStamp[:gen_batch_size-pad_size] = thisTimeStamp
        aux_TrackID[:gen_batch_size-pad_size] = thisTrackID

        thisSampleMapComp = aux_SampleMapComp
        thisSampeHistPath = aux_SampeHistPath
        thisSampeTargetPath = aux_SampeTargetPath
        thisHistAvail = aux_HistAvail
        thisTargetAvail = aux_TargetAvail
        TimeStamp = aux_TimeStamp
        thisTrackID = aux_TrackID
        
        
        

    # Predict
    if base_model:
        predPath = forwardpass_use(thisSampleMapComp, ImageEncModel,  PathDecModel)
    else:
        PathDecModel.reset_states()
        HistEncModel.reset_states()
        predPath = forwardpass_use(thisSampleMapComp, thisSampeHistPath, thisHistAvail, 50,
                                                ImageEncModel, HistEncModel, PathDecModel,
                                                use_teacher_force=False,
                                                increment_net = increment_net,
                                                mruv_guiding = mruv_guiding,
                                                mruv_model = mruv_model,
                                                mruv_model_trainable = mruv_model_trainable)
    
    predPath = predPath.numpy()

    # convert agent coordinates into world offsets
    predPath = predPath[:,:,:2]
    world_from_agents = thisWorldFromAgent.numpy()
    centroids = thisCentroid.numpy()



    
    for idx_sample in range(gen_batch_size-pad_size):
        
        # Save info
        future_coords_offsets_pd.append(transform_points(predPath[idx_sample,:,:], thisWorldFromAgent.numpy()[idx_sample,:,:]) -thisCentroid.numpy()[idx_sample,:])
        timestamps.append(thisTimeStamp[idx_sample].numpy())
        agent_ids.append(thisTrackID[idx_sample])

assert len(agent_ids) == len(validation_dataset), "Validation data size not equal to dataset."
    

Validating: : 100%|██████████| 2960/2960 [1:35:32<00:00,  1.94s/it]


In [29]:
pred_path = '/tf/2020-10-Lyft/prediction-dataset/validate_chopped_100/validation_submission.csv'

write_pred_csv(pred_path,
               timestamps=np.array(timestamps),
               track_ids=np.array(agent_ids, dtype=np.int32),
               coords=np.array(future_coords_offsets_pd),
               confs=None)

In [30]:


metricas_out = compute_metrics_csv(eval_gt_path, pred_path, [neg_multi_log_likelihood, 
                                                             time_displace, 
                                                             rmse, 
                                                             average_displacement_error_mean])

In [31]:
for metric_name, metric_mean in metricas_out.items():
    print(metric_name, metric_mean)

neg_multi_log_likelihood 244.92919855052006
time_displace [0.04934764 0.08579769 0.12089393 0.1561413  0.19210552 0.228615
 0.26647733 0.30567488 0.34602303 0.38794152 0.43244486 0.47885622
 0.52605959 0.5742509  0.62246936 0.67125626 0.71984946 0.76881484
 0.81636672 0.86517557 0.91392045 0.96253291 1.01190729 1.05998885
 1.1075651  1.15481851 1.20301443 1.24916558 1.2981305  1.34444181
 1.39212355 1.43916194 1.48821257 1.53370729 1.58073594 1.62679886
 1.67275219 1.71962976 1.76466566 1.81135897 1.8579979  1.90341346
 1.95021814 1.99750129 2.04603132 2.09103832 2.13543204 2.17920677
 2.22588173 2.27277471]
rmse 1.4403223951224309
average_displacement_error_mean 5.36015060563473
